In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import CRUD module
from AAC_CRUD import AnimalShelterCRUD


###########################
# Data Manipulation / Model
###########################

#Login Credentials
username = "aacuser"
password = "SNHUcs340"
HOST = 'nv-desktop-services.apporto.com'
PORT = 31895
DB = 'AAC'
COL = 'animals'


# Connect to database via CRUD Module
db = AnimalShelterCRUD(username, password, HOST, PORT, DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo image
image_filename = 'GraSalLogo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    # Unique content and logo combined
    html.Div(
        children=[
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     style={'width': '125px'}),
            html.H1('Rescue Animal Dashboard by Amanda Kerr',
                style={'margin': '0'})],
        style={'display': 'flex','align-items': 'center', 'justify-content': 'center', 'gap': '5px'}),
    html.Hr(),
    html.Div([
        html.Label('Filter by Rescue Type:'),     # Advises user how to use filter
        dcc.Dropdown(     # Enables drop down menu
            id='filter-type',
            options=[
                {'label' : 'Water Rescue', 'value' : 'Water'},
                {'label' : 'Mountain and Wilderness Rescue', 'value' : 'Mountain'},
                {'label' : 'Disaster and Individual Rescue', 'value' : 'Disaster'},
                {'label' : 'None', 'value' : 'None'}     # Acts as reset
            ],
            placeholder='Select a rescue type...'
        )
    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
        
                        #Features for interactive data table to make it user-friendly for the client
                         editable=False,     # Disables table edit
                         filter_action="native",     # Enables filtering
                         sort_action="native",     # Enables sorting
                         sort_mode="multi",     # Allows multi-column sorting
                         row_selectable="single",     # Enables row selection
                         row_deletable=False,     # Disables row delete
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",     # Enables pages
                         page_current=0,     # Sets current page to 0
                         page_size=10     # Sets default number of rows per page

    ),
    html.Br(),
    html.Hr(),

    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', 
                'gap': '20px',     # Added small space between graph and map
                'justify-content': 'center',     # Centers map and graph
                'align-items': 'center',     # Aligns map and graph
               }, 
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    df = None 
    
    if filter_type == 'Water' :
        df = pd.DataFrame.from_records(db.read(
            {"$and" : [
                {"$or" : [
                    {'breed' : 'Labrador Retriever Mix'},
                    {'breed' : 'Chesa Bay Rtr'},    #DB lists Chesapeake Bay Retriever as Chesa Bay Retr
                    {'breed' : 'Newfoundland'}]},
                {'sex_upon_outcome' : 'Intact Female'},
                {"$and" : [
                    {'age_upon_outcome_in_weeks' : {"$gt" : 26}},
                    {'age_upon_outcome_in_weeks' : {"$lt" : 156}}
                ]}
            ]}))
    elif filter_type == 'Mountain' : 
        df = pd.DataFrame.from_records(db.read(
            {"$and" : [
                {"$or" : [
                    {'breed' : 'German Shepherd'},
                    {'breed' : 'Alaskan Malamute'},
                    {'breed' : 'Old English Sheepdog'},
                    {'breed' : 'Siberian Husky'},
                    {'breed' : 'Rottweiler'}]},
                {'sex_upon_outcome' : 'Intact Male'},
                {"$and" : [
                    {'age_upon_outcome_in_weeks' : {"$gt" : 26}},
                    {'age_upon_outcome_in_weeks' : {"$lt" : 156}}
                ]}
            ]}))
    elif filter_type == 'Disaster' :
        df = pd.DataFrame.from_records(db.read(
            {"$and" : [
                {"$or" : [
                    {'breed' : 'Doberman Pinsch'},     #DB lists Doberman Pinscher as Doberman Pinsch
                    {'breed' : 'German Shepherd'},
                    {'breed' : 'Golden Retriever'},
                    {'breed' : 'Bloodhound'},
                    {'breed' : 'Rottweiler'}]},
                {'sex_upon_outcome' : 'Intact Male'},
                {"$and" : [
                    {'age_upon_outcome_in_weeks' : {"$gt" : 20}},
                    {'age_upon_outcome_in_weeks' : {"$lt" : 300}}
                ]}
                ]}))
    else: 
        df = pd.DataFrame.from_records(db.read({}))
    
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')
    
    
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Animal Breed Distribution', width=800, height=500)
    fig.update_traces(textposition='inside')
    fig.update_layout(uniformtext_minsize=18, uniformtext_mode='hide') # Fixed size for text
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return 
    elif index is None:
        return 
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:12913/
